In [1]:
# Allow python to import modules in our src folders
import sys
sys.path.append("src/")

from preprocessing.load_dataset import *

In [2]:
characters_df = load_characters_df()
movies_df = load_movies_df()
plot_df = load_plot_df()
tvtropes_tf = load_tvtropes_df()

# I suppose that we do not have cameras in 1010...
# movies_df.iloc[62836] = movies_df.iloc[62836].replace(to_replace='1010-12-02', value='2010-12-02')

In [3]:
characters_df.sample(5)

,wiki_movie_id,freebase_movie_id,release_date,character_name,actor_birth,actor_gender,actor_height,actor_ethnicity,actor_name,release_actor_age,freebase_map_id,freebase_character_id,freebase_actor_id
28889,8326277,/m/026_hqx,2003,NaN,1972-03-24,M,NaN,NaN,Gabriel Olds,30.0,/m/0cg2gvq,NaN,/m/03nn2mn
248542,4451440,/m/0c34n5,2000-03-31,P.E. Teacher,1955-05-27,M,1.780,NaN,Richard Schiff,44.0,/m/0h6cmf0,/m/0hh60d3,/m/04bd8y
77025,3477291,/m/09fllv,1987,NaN,1946-04-12,M,1.854,NaN,Ed O'Neill,40.0,/m/0h28nb4,NaN,/m/039g82
63096,22892307,/m/07scdbc,2010-03-14,Bert Rodriguez,1981-02-17,M,1.780,/m/013b6_,Joseph Gordon-Levitt,29.0,/m/0bj71c3,/m/08mjfdc,/m/05zbm4
214726,10359672,/m/02q9d9w,2007-06-02,NaN,1952-07-20,F,1.600,NaN,Keiko Matsuzaka,54.0,/m/0cvcpwn,NaN,/m/0277x5t


In [4]:
movies_df.sample(5)

,wiki_movie_id,freebase_movie_id,movie_name,movie_release_date,box_office,movie_runtime,movie_languages,movie_countries,movie_genres
17401,14647019,/m/03grrgt,Nan of Music Mountain,1917-12-12,NaN,50.0,"{'/m/06ppq': 'Silent film', '/m/02h40lc': 'Eng...",{'/m/09c7w0': 'United States of America'},"{'/m/06ppq': 'Silent film', '/m/0219x_': 'Indi..."
62433,3745060,/m/09yq12,"I, a Man",1967-08-24,NaN,99.0,{'/m/02h40lc': 'English Language'},{'/m/09c7w0': 'United States of America'},"{'/m/0424mc': 'Experimental film', '/m/07s9rl0..."
62685,22645209,/m/05zsts6,De Stilte van het Naderen,2000,NaN,104.0,{'/m/02bv9': 'Dutch Language'},{'/m/059j2': 'Netherlands'},"{'/m/01jfsb': 'Thriller', '/m/02l7c8': 'Romanc..."
1176,9002666,/m/027t0d9,Il diavolo,1963,NaN,103.0,{'/m/02bjrlw': 'Italian Language'},{'/m/03rjj': 'Italy'},"{'/m/0gsy3b': 'Sex comedy', '/m/03p5xs': 'Come..."
27450,6369028,/m/0g2t9q,All About Ah Long,1989,NaN,96.0,"{'/m/07zrf': 'Vietnamese Language', '/m/0459q4...",{'/m/03h64': 'Hong Kong'},"{'/m/02h8pkk': 'Family Drama', '/m/07s9rl0': '..."


In [5]:
print(f"Number of movies loaded: {len(movies_df)}")

Number of movies loaded: 81741


In [6]:
plot_df.sample(5)

,wiki_movie_id,plot_summary
9020,9409095,Jericho Hudson is a street smart kid with a gi...
11481,9310559,"It is divided into ten scenes, each of which d..."
3708,32549690,"A young woman, Janice, is living with her rest..."
22252,31304530,Young noblewoman Elisabeth von Salmenau encou...
36304,28559151,"In Gdansk, Poland, in 1989, Alexander Reschke ..."


In [7]:
tvtropes_tf.sample(5)

,trope_name,character_data
442,stoner,"{'char': 'Thadeous', 'movie': 'Your Highness',..."
429,slacker,"{'char': 'John Winger', 'movie': 'Stripes', 'i..."
178,dirty_cop,"{'char': 'Colin Sullivan', 'movie': 'The Depar..."
173,dean_bitterman,"{'char': 'Ed Rooney', 'movie': 'Ferris Bueller..."
417,romantic_runnerup,"{'char': 'Proteus', 'movie': 'Sinbad: Legend o..."


In [8]:
# extract years from the release date of movies
movies_df.loc[:, 'movie_release_year'] = movies_df['movie_release_date'].astype(str).str[:4]
movies_df = movies_df[movies_df['movie_release_year'].str.contains("nan")==False]
movies_df['movie_release_year'] = movies_df['movie_release_year'].astype(int)


In [9]:
wrong_dates = movies_df[movies_df['movie_release_year'] < 1800].index
# there is only one movie that has the release date 1010, so we can change 1010 to 2010
movies_df.loc[wrong_dates, 'movie_release_year']= 2010
movies_df.loc[wrong_dates, 'movie_release_date'] ='2010-12-02'

## Perfomance of the movie in terms of revenue

In [10]:
# how many missing values in the column box_office
percentage_missing = movies_df['box_office'].isna().sum() / len(movies_df) * 100
print("{:.2f}% values for the box office revenue are missing.".format(percentage_missing))

88.87% values for the box office revenue are missing.


In [11]:
# There is no revenue from IMDB dataset ?
# https://developer.imdb.com/non-commercial-datasets/

## Extract ratings from the IMDB rating dataset

file: title.rating.tsv
-    tconst: unique identifier of the movie
-    averageRating: average of user ratings
-    numVotes: number of ratings submitted for the movie


In [17]:
# REVIEWS_METADATA = "title.ratings.tsv/data.tsv"
imdb_ratings_df = load_imdb_ratings()
imdb_names_df = load_imdb_basics()
# imdb_crew_df = load_imdb_crew()
# imdb_akas_df = load_imdb_akas()
# imdb_episodes_df = load_imdb_episodes()
# imdb_principals_df = load_imdb_principals()

In [13]:
# Extract the movies from the title.basics imbd dataset
imdb_names_df.head()


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,\N,45,Romance
144,tt0000147,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0,1897,\N,100,"Documentary,News,Sport"
498,tt0000502,movie,Bohemios,Bohemios,0,1905,\N,100,\N
570,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,\N,70,"Action,Adventure,Biography"
587,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,\N,90,Drama


In [14]:
# merge with ratings
imdb_ratings_meta = imdb_ratings_df.merge(imdb_names_df, on='tconst', how='inner')
imdb_ratings_meta.head()

,tconst,averageRating,numVotes,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000009,5.3,207,movie,Miss Jerry,Miss Jerry,0,1894,\N,45,Romance
1,tt0000147,5.3,482,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0,1897,\N,100,"Documentary,News,Sport"
2,tt0000502,4.1,15,movie,Bohemios,Bohemios,0,1905,\N,100,\N
3,tt0000574,6.0,850,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,\N,70,"Action,Adventure,Biography"
4,tt0000591,4.4,20,movie,The Prodigal Son,L'enfant prodigue,0,1907,\N,90,Drama


## Merge the CMU dataset with the IMDB

In [15]:
merged_df = movies_df.merge(imdb_ratings_meta, left_on='movie_name', right_on = 'primaryTitle', how='inner')
print("Size of merged dataset:", merged_df.shape[0])
merged_df.head()

Size of merged dataset: 89255


,wiki_movie_id,freebase_movie_id,movie_name,movie_release_date,box_office,movie_runtime,movie_languages,movie_countries,movie_genres,movie_release_year,...,averageRating,numVotes,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,{'/m/02h40lc': 'English Language'},{'/m/09c7w0': 'United States of America'},"{'/m/01jfsb': 'Thriller', '/m/06n90': 'Science...",2001,...,4.9,56673,movie,Ghosts of Mars,Ghosts of Mars,0,2001,\N,98,"Action,Horror,Sci-Fi"
1,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,{'/m/05f_3': 'Norwegian Language'},{'/m/05b4w': 'Norway'},"{'/m/0lsxr': 'Crime Fiction', '/m/07s9rl0': 'D...",1988,...,5.6,40,movie,Brun bitter,Brun bitter,0,1988,\N,83,"Crime,Drama"
2,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,{'/m/04306rv': 'German Language'},{'/m/0345h': 'Germany'},{'/m/07s9rl0': 'Drama'},1983,...,6.0,616,movie,A Woman in Flames,Die flambierte Frau,0,1983,\N,106,Drama
3,13696889,/m/03cfc81,The Gangsters,1913-05-29,NaN,35.0,"{'/m/06ppq': 'Silent film', '/m/02h40lc': 'Eng...",{'/m/09c7w0': 'United States of America'},"{'/m/02hmvc': 'Short Film', '/m/06ppq': 'Silen...",1913,...,5.6,33,movie,The Gangsters,Les truands,0,1957,\N,105,"Comedy,Crime"
4,23070193,/m/064mmt5,The Gangsters,1956,NaN,NaN,{'/m/064_8sq': 'French Language'},{'/m/0f8l9c': 'France'},{'/m/05p553': 'Comedy film'},1956,...,5.6,33,movie,The Gangsters,Les truands,0,1957,\N,105,"Comedy,Crime"
